In [1]:
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point
import json
from shapely import to_geojson
from shapely.geometry import shape
from networkx.readwrite import json_graph
import folium

In [2]:
def graph_to_geojson(G, path):
    """
    Serialize NetworkX graph G (with Shapely geometries in node attrs)
    to a JSON file at 'path', converting geometries via to_geojson.
    """
    # 1. Extract node-link data
    data = json_graph.node_link_data(G)
    # 2. Replace each geometry attribute with a GeoJSON dict
    for node in data["nodes"]:
        geom = node.get("geometry")
        if geom is not None:
            # to_geojson returns a JSON string; parse it
            geojson_str = to_geojson(geom, indent=None)
            node["geometry"] = json.loads(geojson_str)
    # 3. Dump to file
    with open(path, "w") as f:
        json.dump(data, f, indent=2)


def graph_from_geojson(path, directed=False, multigraph=True):
    """
    Read a JSON file at 'path' produced by graph_to_geojson and
    reconstruct the original NetworkX graph with Shapely geometries.
    """
    # 1. Load the raw node-link dict
    with open(path) as f:
        data = json.load(f)

    # 2. Convert each node's GeoJSON dict back into a Shapely geometry
    for node_obj in data.get("nodes", []):
        geom_dict = node_obj.get("geometry")
        if isinstance(geom_dict, dict) and "type" in geom_dict:
            node_obj["geometry"] = shape(geom_dict)  # inverse of to_geojson :contentReference[oaicite:1]{index=1}

    # 3. (Optional) If you stored edge geometries similarly, undo those too
    for edge_obj in data.get("links", data.get("edges", [])):
        geom_dict = edge_obj.get("geometry")
        if isinstance(geom_dict, dict) and "type" in geom_dict:
            edge_obj["geometry"] = shape(geom_dict)

    # 4. Rebuild the NetworkX graph (with all attrs, including restored geometries)
    G = json_graph.node_link_graph(
        data,
        directed=directed,
        multigraph=multigraph
    )  # rebuilds Graph from node-link format :contentReference[oaicite:2]{index=2}

    return G


In [8]:
G = graph_from_geojson(path="graphs/lva-railroad-network.json")
m = folium.Map(location=[57.0, 25.0], zoom_start=7)

for node1, node2 in G.edges():
    if node1 in G.nodes and node2 in G.nodes and 'geometry' in G.nodes[node1] and 'geometry' in G.nodes[node2]:
        geom1 = G.nodes[node1]['geometry']
        geom2 = G.nodes[node2]['geometry']
        if isinstance(geom1, Point) and isinstance(geom2, Point):
            lat1, lon1 = geom1.y, geom1.x
            lat2, lon2 = geom2.y, geom2.x
            folium.PolyLine([[lat1, lon1], [lat2, lon2]], color='blue', weight=1).add_to(m)
        else:
            print(f"Warning: Skipping edge between {node1} and {node2} due to non-Point geometry.")
    else:
        print(f"Warning: Skipping edge between {node1} and {node2} because one or both nodes are missing geometry information.")

# Mark the nodes (stations) with red dots
for node, data in G.nodes(data=True):
    if 'geometry' in data and isinstance(data['geometry'], Point):
        folium.CircleMarker(
            location=[data['geometry'].y, data['geometry'].x],
            radius=1,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7
        ).add_to(m)

# Save or display the map
m.save(f'vizualizations/lva_railroads.html')

In [ ]:
country_codes: list[str] = [# "svk", 
                            # "che", 
                            # "aut",
                            # "cze", 
                             "bel", "blr", "nld", "lva"] 
geographical_centers: list[list[float]] = [
    #[48.7, 19.5],            # Slovakia (svk)
    # [46.81, 8.2],         # Switzerland (che)
    #[47.3333, 13.3333],   # Austria (aut)
    #[49.75, 15.5],        # Czechia (cze)
    [50.64111, 4.66806],  # Belgium (bel)
    [53.52904, 28.04497], # Belarus (blr)
    [52.3125, 5.5486],    # Netherlands (nld)
    [57.0, 25.0]          # Latvia (lva)
]
for idx, country in enumerate(country_codes):
    print(country)
    print(geographical_centers[idx])

bel
[50.64111, 4.66806]
blr
[53.52904, 28.04497]
nld
[52.3125, 5.5486]
lva
[57.0, 25.0]


In [ ]:
country_codes: list[str] = [#"aut",
                            # "cze", "che", "svk",
                             "bel", "blr", "nld", "lva"]  
geographical_centers: list[list[float]] = [
    #[47.3333, 13.3333],   # Austria (aut)
    #[49.75, 15.5],        # Czechia (cze)
    [50.64111, 4.66806],  # Belgium (bel)
    [53.52904, 28.04497], # Belarus (blr)
    [52.3125, 5.5486],    # Netherlands (nld)
    [57.0, 25.0]          # Latvia (lva)
]
for idx, country in enumerate(country_codes):
    # loading data
    stations_gdf =  gpd.read_file(f"data/hotosm_{country}_railways_points_geojson.geojson")
    stations_gdf = stations_gdf[stations_gdf['railway'] == 'station']
    tracks_gdf = gpd.read_file(f"data/hotosm_{country}_railways_lines_geojson.geojson")
    tracks_gdf = tracks_gdf[tracks_gdf['railway'] == 'rail'] 
    # graph creation
    G = nx.Graph()

    for index, row in stations_gdf.iterrows():
        if row["name"] and row["geometry"]:
            G.add_node(row["name"], geometry=row["geometry"], id=row["osm_id"])
        
    # Add edges based on tracks
    for idx, track in tracks_gdf.iterrows():
        start_point = Point(track.geometry.coords[0])
        end_point = Point(track.geometry.coords[-1])

        # Find nearest station to start
        start_station = stations_gdf.iloc[stations_gdf.geometry.distance(start_point).idxmin()]
        start_city = start_station['name']

        # Find nearest station to end
        end_station = stations_gdf.iloc[stations_gdf.geometry.distance(end_point).idxmin()]
        end_city = end_station['name']

        # Add edge if cities are different
        if start_city != end_city and start_city and end_city:  # Ensure cities exist
            G.add_edge(start_city, end_city)
    

    graph_to_geojson(G=G, path=f"graphs/{country}-railroad-network.json")
    try:
        m = folium.Map(location=geographical_centers[idx], zoom_start=7)

        for node1, node2 in G.edges():
            if node1 in G.nodes and node2 in G.nodes and 'geometry' in G.nodes[node1] and 'geometry' in G.nodes[node2]:
                geom1 = G.nodes[node1]['geometry']
                geom2 = G.nodes[node2]['geometry']
                if isinstance(geom1, Point) and isinstance(geom2, Point):
                    lat1, lon1 = geom1.y, geom1.x
                    lat2, lon2 = geom2.y, geom2.x
                    folium.PolyLine([[lat1, lon1], [lat2, lon2]], color='blue', weight=1).add_to(m)
                else:
                    print(f"Warning: Skipping edge between {node1} and {node2} due to non-Point geometry.")
            else:
                print(f"Warning: Skipping edge between {node1} and {node2} because one or both nodes are missing geometry information.")

        # Mark the nodes (stations) with red dots
        for node, data in G.nodes(data=True):
            if 'geometry' in data and isinstance(data['geometry'], Point):
                folium.CircleMarker(
                    location=[data['geometry'].y, data['geometry'].x],
                    radius=1,
                    color='red',
                    fill=True,
                    fill_color='red',
                    fill_opacity=0.7
                ).add_to(m)

        # Save or display the map
        m.save(f'vizualizations/{country}_railroads.html')
    except:
        pass

C:\Users\pazera\AppData\Local\Temp\ipykernel_22832\906148296.py:31: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  start_station = stations_gdf.iloc[stations_gdf.geometry.distance(start_point).idxmin()]
C:\Users\pazera\AppData\Local\Temp\ipykernel_22832\906148296.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  end_station = stations_gdf.iloc[stations_gdf.geometry.distance(end_point).idxmin()]
C:\Users\pazera\AppData\Local\Temp\ipykernel_22832\906148296.py:31: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  start_station = stations_gdf.iloc[stations_gdf.geometry.distanc

In [7]:
from geopy.distance import geodesic
country_codes: list[str] = ["aut", "cze", "che", "svk",
                             "bel", "blr", "nld", "lva"]  
for c_c in country_codes:
    G = graph_from_geojson(path=f"graphs/{c_c}-railroad-network.json")
    for u, v in G.edges():
        # Fallback to straight-line distance using node geometries
        point_u = G.nodes[u]['geometry']
        point_v = G.nodes[v]['geometry']
        coords_u = point_u.coords[0]
        coords_v = point_v.coords[0]
        latlon_u = (coords_u[1], coords_u[0])
        latlon_v = (coords_v[1], coords_v[0])
        total_distance = geodesic(latlon_u, latlon_v).meters
        
        G[u][v]['distance'] = total_distance        
    graph_to_geojson(G=G, path=f"graphs_with_distance/{c_c}-railroad-network.json")

c:\Users\pazera\anaconda3\envs\starter\Lib\site-packages\networkx\readwrite\json_graph\node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(
c:\Users\pazera\anaconda3\envs\starter\Lib\site-packages\networkx\readwrite\json_graph\node_link.py:142: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(


In [35]:
G = graph_from_geojson(path=f"graphs_with_distance/svk-railroad-network.json")


c:\Users\pazera\anaconda3\envs\starter\Lib\site-packages\networkx\readwrite\json_graph\node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


In [36]:
stations = list(G.nodes())
for station in stations:
    
    print(station)

Spišská Belá
Podhorany pri Kežmarku
Podolínec
Spišská Belá zastávka
Bušovce
Hronec
Devínska Nová Ves
Brezno
Malacky
Čaňa
Krásna nad Hornádom
Barca
Malé Straciny
Stará Ľubovňa
Hodejov
Červená Skala
Rimavská Seč
Blhovce
Süttő
Piszke
Balnica
Pohronský Ruskov
Likier
Pribeník
Budkovce ŠRT
Michaľany
Veľké Kapušany
Vojany ŠRT
Čierna nad Tisou
Rožňava
Plavnica
Kittsee
Kunova Teplica
Štítnik
Veľké Leváre
Sekule
Svit
Štrkovec
Ochtiná
Hnilec
Vlachovo
Dobšiná
Nižná Slaná
Muráň
Mlynky
Betliar
Gemerská Hôrka
Jelšava
Martin
Tanečník
Sedlo Beskyd
Čremošné
Radvaň
Počkaj
Kostiviarska
Harmanec jaskyňa
Jablonov nad Turňou
Spišské Podhradie
Hrhov
Moldava nad Bodvou
Švedlár
Pstruša
Slovenská Ľupča
Dobrá Niva
Vpred
Poltár
Banská Belá
Ipeľský Sokolec
Čata
Zohor
Dlhé nad Cirochou
Hontianske Nemce
Strážske
Hrinište
Kúty
Štúrovo
Štúrovo tranzitná skupina
Kysucké Nové Mesto
Žihárec
Rimavské Zalužany
Rimavská Baňa
Pribeta
Strekov
Krásno nad Kysucou
Dvory nad Žitavou
Košice
Čadca
Soľ
Drienovská Nová Ves
Gbely
Holíč

# deleted nodes Slovakia

In [37]:
from networkx import NetworkXError
stations_to_remove = ["Vydrovo - Koliba", "Balnica", "Starý Smokovec", "Hrebienok", "Štrbské Pleso OŽ", "Štrbské Pleso", "Ružomberok-Zápalkáreň",
                      "Ružomberok - Malé Nádražie","Chmúra", "Tanečník", "Sedlo Beskyd", "Bratislava - Hlavná stanica"]
for node in stations_to_remove:
    try:
        G.remove_node(node)
    except NetworkXError:
        pass

# Added stations Slovakia

In [38]:
G.add_node("Bratislava - Hlavná stanica", geometry=Point(17.10416625, 48.15499938))
G.add_node("Bratislava - Vinohrady", geometry=Point(17.13387381009982, 48.18698557873771))

# Added connections Slovakia


In [39]:
from geopy.distance import geodesic
edges_list = [("Malé Straciny", "Lučenec"), ("Hronec zlievareň", "Čierny Balog"), ("Petrovce nad Laborcom", "Michalovce"),
              ("Vpred", "Čermeľ"), ("Alpinka", "Vpred"), ("Bratislava - Hlavná stanica", "Bratislava - Vinohrady"), 
              ("Bratislava - Vinohrady", "Sládkovičovo"), ("Bratislava-Lamač", "Bratislava - Hlavná stanica")]
for u, v in edges_list:
    G.add_edge(u, v)
    point_u = G.nodes[u]['geometry']
    point_v = G.nodes[v]['geometry']
    coords_u = point_u.coords[0]
    coords_v = point_v.coords[0]
    latlon_u = (coords_u[1], coords_u[0])
    latlon_v = (coords_v[1], coords_v[0])
    total_distance = geodesic(latlon_u, latlon_v).meters
    G[u][v]['distance'] = total_distance        



In [40]:
m = folium.Map(location=[48, 17], zoom_start=7)

for node1, node2 in G.edges():
    if node1 in G.nodes and node2 in G.nodes and 'geometry' in G.nodes[node1] and 'geometry' in G.nodes[node2]:
        geom1 = G.nodes[node1]['geometry']
        geom2 = G.nodes[node2]['geometry']
        if isinstance(geom1, Point) and isinstance(geom2, Point):
            lat1, lon1 = geom1.y, geom1.x
            lat2, lon2 = geom2.y, geom2.x
            folium.PolyLine([[lat1, lon1], [lat2, lon2]], color='blue', weight=1).add_to(m)
        else:
            print(f"Warning: Skipping edge between {node1} and {node2} due to non-Point geometry.")
    else:
        print(f"Warning: Skipping edge between {node1} and {node2} because one or both nodes are missing geometry information.")

# Mark the nodes (stations) with red dots
for node, data in G.nodes(data=True):
    if 'geometry' in data and isinstance(data['geometry'], Point):
        folium.CircleMarker(
            location=[data['geometry'].y, data['geometry'].x],
            radius=1,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7
        ).add_to(m)

# Save or display the map
m.save(f'vizualizations/svk_railroads.html')

In [41]:
graph_to_geojson(G=G, path="selected_fixed_graphs/svk-railroad-network.json" )

c:\Users\pazera\anaconda3\envs\starter\Lib\site-packages\networkx\readwrite\json_graph\node_link.py:142: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(
